In [9]:
# load modules
import folium
import pandas

In [10]:
# list of avaliable objects and methods from the Folium module
print(dir(folium))

['Circle', 'CircleMarker', 'ClickForMarker', 'ColorLine', 'ColorMap', 'CssLink', 'CustomIcon', 'Div', 'DivIcon', 'Element', 'FeatureGroup', 'Figure', 'FitBounds', 'GeoJson', 'GeoJsonTooltip', 'Html', 'IFrame', 'Icon', 'JavascriptLink', 'LatLngPopup', 'LayerControl', 'LinearColormap', 'Link', 'MacroElement', 'Map', 'Marker', 'PolyLine', 'Polygon', 'Popup', 'Rectangle', 'RegularPolygonMarker', 'StepColormap', 'TileLayer', 'Tooltip', 'TopoJson', 'Vega', 'VegaLite', 'WmsTileLayer', '__all__', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', '__version__', '_version', 'absolute_import', 'branca', 'division', 'features', 'folium', 'map', 'print_function', 'raster_layers', 'utilities', 'vector_layers']


In [11]:
# load data with the list of volcanoes 
data = pandas.read_csv("Volcanoes_USA.txt")
print(data.columns)
lat = data["LAT"]
lon = data["LON"]
name = data["NAME"]
elev = data["ELEV"]


Index(['VOLCANX020', 'NUMBER', 'NAME', 'LOCATION', 'STATUS', 'ELEV', 'TYPE',
       'TIMEFRAME', 'LAT', 'LON'],
      dtype='object')


In [12]:
# create a map 
map = folium.Map(location=[38.261722, -98.413330], zoom_start=6,tiles="Mapbox Bright")

# set up a feature group
fg = folium.FeatureGroup(name="My Map")

# add to the feature group all volcanoes, and set up a color of a marker according to an elevation
for lt, ln, nm, el in zip(lat,lon,name,elev):
    # add popup message with the name corresponding to each volcano
    popup = folium.Popup(nm, parse_html=True)
    if el < 1000.0:
        fg.add_child(folium.Marker(location=[lt,ln],popup=popup, icon=folium.Icon(color='blue')))
    elif el < 2000.0:
        fg.add_child(folium.Marker(location=[lt,ln],popup=popup, icon=folium.Icon(color='green')))
    else:
        fg.add_child(folium.Marker(location=[lt,ln],popup=popup, icon=folium.Icon(color='red')))

# add the feature group to the map
map.add_child(fg)

# save map into an .html file
map.save("Map1.html")

In [13]:
# define color producer function corresponding to an elevation of a volcano
def color_producer(elevation):
    if elevation < 1000:
        return 'green'
    elif 1000 <= elevation < 3000:
        return 'orange'
    else:
        return 'red'

# create another map
map2 = folium.Map(location=[38.261722, -98.413330], zoom_start=6,tiles="Mapbox Bright")

# set up the feature group "Volcanoes"
fg2 = folium.FeatureGroup(name="Volcanoes")

# add volcanoes to the feature group "Volcanoes" as Circle Markers
for lt, ln, el in zip(lat, lon, elev):
    cm = folium.CircleMarker(location=[lt, ln],
                            radius = 6,
                            popup=str(el)+" m",
                            fill=True, # Set fill to True
                            fill_color=color_producer(el),
                            color = 'grey',
                            fill_opacity=0.7)
    fg2.add_child(cm)

# set up the feature group "Population"
fg3 = folium.FeatureGroup(name="Population")

# color countries according to the population 
fg3.add_child(folium.GeoJson(data=open("world.json","r", encoding='utf-8-sig').read(), 
                             style_function=lambda x: {'fillColor':'green' if x['properties']['POP2005'] < 10000000 
                                                       else 'orange' if x['properties']['POP2005'] < 50000000 else 'red'}))
# add feature groups "Volcanoes" and "Population" to the map
map2.add_child(fg2)
map2.add_child(fg3)

# add layer control allowing to turn on and off the added feature groups
map2.add_child(folium.LayerControl())

# save the map into an .html file
map2.save("Map2.html")